<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe61622](https://jckantor.github.io/cbe61622);
content is available [on Github](https://github.com/jckantor/cbe61622.git).*


<!--NAVIGATION-->
< [7.4 Stepper Motor Control via I2C](https://jckantor.github.io/cbe61622/07.04-Stepper_Motor_Control_via_I2C.html) | [Contents](toc.html) | [8.0 Raspberry Pi](https://jckantor.github.io/cbe61622/08.00-Raspberry-Pi.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/07.10-Wifi-with-ESP-01.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe61622/07.10-Wifi-with-ESP-01.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 7.10 Wifi

Raspberry Pi Pico

* [Raspberry Pi Pico Pinout Diagram](https://datasheets.raspberrypi.com/pico/Pico-R3-A4-Pinout.pdf)


ESP-01 and ESP8266

* [ESP8266 Wikipedia article](https://en.wikipedia.org/wiki/ESP8266)

* [ESP8266 Non-OS AT Instruction Set](https://www.espressif.com/sites/default/files/documentation/4a-esp8266_at_instruction_set_en.pdf) Espressif's documentation of the AT command set.

* [ESP8266 AT Commands from Electronics Hub](https://www.electronicshub.org/esp8266-at-commands/) A tutorial summary of the AT command set.

* [AT Command Reference](http://room-15.github.io/blog/2015/03/26/esp8266-at-command-reference/) A blog description of all known AT commands implemented in the default firmware of the ESP8266 mcu.

Examples
* [Webserver Using Pi Pico and ESP01](https://www.instructables.com/Webserver-Using-Pi-Pico-and-ESP01/)

* [Connect ESP-01S (ESP8266) to Raspberry Pi Pico/MicroPython, using AT Command](http://helloraspberrypi.blogspot.com/2021/02/connect-esp-01s-esp8266-to-raspberry-pi.html)

* [Using AT commands on the ESP8266](https://learn.circuit.rocks/using-at-commands-on-the-esp8266) 

* [Pico/MicroPython + ESP-01S (AT Command) act as TCP Client connect to Raspberry Pi/Python TCP Server](http://helloraspberrypi.blogspot.com/2021/02/picomicropython-esp-01s-at-command-act.html) A bare bones example of communicating with the ESP8266 using AT commands.

* [Raspberry Pi Pico Micropython ESP8266 Library](https://github.com/noyelseth/rpi-pico-micropython-esp8266-lib) MicroPython classes to access ESP8266 by AT commands, and to parse HTTP response 
after an HTTP Post/Get operation.

Video
* [Raspberry Pi Pico gets WiFi via an ESP01 and UART](https://youtu.be/02yfpA3h_w8https://youtu.be/02yfpA3h_w8)

## 7.10.1 ESP01 class

In [165]:
%serialconnect

from machine import UART, Pin
import time

UART_DELAY_MS = 500

class ESP_Error(Exception):
    """Raise custom exceptions for ESP module."""
    pass


class ESP01():
    
    def __init__(self, uart_port=0, tx=16, rx=17, verbose=False):
        self.txPin = Pin(tx)
        self.rxPin = Pin(rx)
        self.uart = UART(0, baudrate=115200, tx=self.txPin, rx=self.rxPin, timeout=10)
        self.verbose = verbose
        if self.verbose:
            self.echo_on()
            print(self.send("AT"))
        else:
            self.echo_off()

    def send(self, cmd, timeout=UART_DELAY_MS):
        if self.verbose:
            print("+"*20 + "\n command:\t", cmd)
            print(" timeout:\t", timeout, "ms")

        # write AT command
        self.uart.write(cmd + "\r\n")
        
        # wait for data to appear in the buffer
        toc = time.ticks_ms()
        while time.ticks_ms() - toc < timeout:
            if self.uart.any():
                break
                
        # read buffer
        rxData = bytes()
        toc = time.ticks_ms()
        while True:
            if (time.ticks_ms() - toc) > timeout:
                break
            if self.uart.any():
                rxData += self.uart.read(1)
                toc = time.ticks_ms()
                
        # report response
        try:
            rxData = rxData.decode()
        except UnicodeError:
            if self.verbose:
                print("Unicode error encountered, returning byte string.")
        if self.verbose:
            print("response:\n", rxData)
            print("-"*20)
            
        if "OK" in rxData:
            status = "OK"
        elif "FAIL" in rxData:
            status = "FAIL"
        elif "busy p..." in rxData:
            status = "BUSY"
        elif "ERROR" in rxData:
            status = "ERROR"
        else:
            status = "UNKNOWN"
            
        if status != "OK":
            print("status = ", status)
            
        return rxData
            
        
    def echo_off(self):
        """Turn off the echo feature of the ESP-01."""
        return self.send("ATE0")
    
    def echo_on(self):
        """Turn on the echo feature of the ESP-01."""
        return self.send("ATE1")
    
    def restart(self):
        """Restart ESP8266 module."""
        return self.send("AT+RST", 500)
    
    def get_version(self):
        """Return firmware version."""
        return self.send("AT+GMR")
    
    def set_wifi_mode(self, mode=1):
        return self.send("AT+CWMODE_CUR=" + str(mode))
        
    def get_wifi_mode(self):
        response = self.send("AT+CWMODE_CUR?")
        if response is not None:
            if "1" in response:
                return "STA"
            elif "2" in response:
                return "SoftAP"
            elif "3" in response:
                return "SoftAP +  STA"
            else:
                raise ESP_Error("Unrecognized WiFi mode returned: \n" + response)
        return None
        
    def get_APs(self):
        return self.send("AT+CWLAP", 5000)
    
    def connect(self, ssid, pwd):
        cmd = "AT+CWJAP=" + '"' + ssid + '"' + ',' + '"' + pwd + '"'
        return self.send(cmd.encode(), timeout=10000)
    
    def get_IP_address(self):
        return self.send("AT+CIFSR")
    
    def set_connection_mode(self, mode=1):
        return self.send("AT+CIPMUX=1")
    
    def webserver(self):
        return self.send("AT+CIPSERVER=1,80", 2000)

serial exception on close write failed: [Errno 6] Device not configured
Found serial ports: /dev/cu.usbmodem14401, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14401 --baud=115200 
Ready.


In [166]:
# startup

esp = ESP01(verbose=True)
esp.restart()
esp.get_version()
esp.set_wifi_mode(1)

# connection
#esp.connect("Wilder\u2019s Edge", "FreyaWatson")
print(esp.connect("ND-guest", ""))
esp.set_connection_mode()
esp.get_IP_address()

# webserver

def read_http_request():
    req = ""
    while True:
        recv = bytes()
        line = 0
        if esp.uart.any():
            recv = esp.uart.readline()
        recv = recv.decode()
        recv = recv.replace("\r\n", "\n")
        if len(recv) == 1:
            break
        else:
            if len(recv) > 0:
                req += recv
    
    lines = req.split("\n")
    if "+IPD" in lines[0]:
        print("http request received")
    else:
        print("AT response received")
    for k, line in enumerate(lines):
         print(k, ":", line)
        
        
def send_line(msg):
    esp.send(f"AT+CIPSEND=0,{len(msg)+2}", 10)
    esp.send(msg, 10)
    
def send_http_response(n):
    send_line("HTTP/1.1 200 OK")
    send_line("Content-type:text/html")
    send_line("Connection: close")
    send_line("");
    send_line("<html><body>")
    send_line("<h1>Hello World!</h1>")
    send_line(f"message {n}")
    send_line("</body></html>")
    esp.send("AT+CIPCLOSE=0")
    
esp.webserver()
recv_buf = bytes()
toc = time.ticks_ms()
n = 0
while True:
    read_http_request()
    send_http_response(n)
    n += 1
    

++++++++++++++++++++
 command:	 ATE1
 timeout:	 500 ms
Unicode error encountered, returning byte string.
response:
 b'\xfeATE1\r\r\n\r\nOK\r\n'
--------------------
++++++++++++++++++++
 command:	 AT
 timeout:	 500 ms
response:
 AT

OK

--------------------
AT

OK

++++++++++++++++++++
 command:	 AT+RST
 timeout:	 500 ms
Unicode error encountered, returning byte string.
response:
 b'AT+RST\r\r\n\r\nOK\r\nWIFI DISCONNECT\r\n\r\n ets Jan  8 2013,rst cause:2, boot mode:(3,7)\r\n\r\nload 0x40100000, len 27728, room 16 \r\ntail 0\r\nchksum 0x2a\r\nload 0x3ffe8000, len 2124, room 8 \r\ntail 4\r\nchksum 0x07\r\nload 0x3ffe8850, len 9276, room 4 \r\ntail 8\r\nchksum 0xba\r\ncsum 0xba\r\n\x8c\xe3No\x83l\x8e\x8cl\x130\xec\x0c\x88\x80\x0c\x8es\x88\x03\x8c\xe3No\x83l\x8e\x8fd\x130\xe4\x0c\x88\x80\x0c\x8es\x88\x03\x8c\xe3No\x83l\x8e\x98l\x130\xec\x0c\x88\x80l\x8es\x88\x03s\x88\x03\x8f\t\x907\xff\r\nready\r\n'
--------------------
++++++++++++++++++++
 command:	 AT+GMR
 timeout:	 500 ms
response:
 A

In [124]:
    if esp.uart.any():
        recv_buf += esp.uart.readline()
    recv = recv_buf.decode()
    if len(recv) > 0:
        print(recv)
        recv_buf = bytes()
        if "+IPD" in recv:
            time.sleep(1)
            sendline("HTTP/1.1 200 OK")
            sendline("Content-type:text/html")
            sendline("Connection: close")
            sendline("")
            sendline("<h1>Hello World!</h1>")
            sendline(f"time = {(time.ticks_ms() - toc)/1000} seconds")
            esp.send("AT+CIPCLOSE=0")
            time.sleep(1)
            
            

++++++++++++++++++++
 command:	 ATE1
 timeout:	 500 ms
Unicode error encountered, returning byte string.
response:
 b'\xfeATE1\r\r\n\r\nOK\r\n'
--------------------
++++++++++++++++++++
 command:	 AT
 timeout:	 500 ms
response:
 AT

OK

--------------------
AT

OK

++++++++++++++++++++
 command:	 AT+RST
 timeout:	 500 ms
Unicode error encountered, returning byte string.
response:
 b'AT+RST\r\r\n\r\nOK\r\nWIFI DISCONNECT\r\n\r\n ets Jan  8 2013,rst cause:2, boot mode:(3,7)\r\n\r\nload 0x40100000, len 27728, room 16 \r\ntail 0\r\nchksum 0x2a\r\nload 0x3ffe8000, len 2124, room 8 \r\ntail 4\r\nchksum 0x07\r\nload 0x3ffe8850, len 9276, room 4 \r\ntail 8\r\nchksum 0xba\r\ncsum 0xba\r\n\x8c\xe3No\x83l\x8e\x8cd\x130\xe4\x0c\x88\x80\x0c\x8es\x88\x03\x8c\xe3No\x83d\x8e\x8fl\x130\xec\x0c\x88\x80\x0c\x8es\x88\x03\x8c\xe3No\x83l\x8e\x98l\x130\xec\x0c\x88\x80l\x8es\x88\x03s\x88\x03\x8f\t\x907\xff\r\nready\r\n'
--------------------
++++++++++++++++++++
 command:	 AT+GMR
 timeout:	 500 ms
response:
 A

In [42]:
print(esp.get_wifi_mode())

--------------------
 command:	 AT+CWMODE_CUR?
 timeout:	 500 ms
response:	 AT+CWMODE_CUR?
+CWMODE_CUR:1

OK

--------------------
STA


In [2]:
      
esp = ESP01S(verbose=True)
#print(esp.restart())
print(esp.version())
print(esp.set_wifi_mode(1))
print(esp.get_wifi_mode())
print(esp.send("AT+CWJAP?"))

print(esp.connect_wifi("Wilder\u2019s Edge", "FreyaWatson"))

Found serial ports: /dev/cu.usbmodem14101, /dev/cu.Bluetooth-Incoming-Port 
Connecting to --port=/dev/cu.usbmodem14101 --baud=115200 
Ready.
--------------------
 command:	 ATE1
 timeout:	 500 ms
response:	 b'ATE1\r\r\n\r\nOK\r\n'
--------------------
--------------------
 command:	 AT
 timeout:	 500 ms
response:	 b'AT\r\r\n\r\nOK\r\n'
--------------------
AT

OK

--------------------
 command:	 AT+GMR
 timeout:	 500 ms
response:	 b'AT+GMR\r\r\nAT version:1.1.0.0(May 11 2016 18:09:56)\r\nSDK version:1.5.4(baaeaebb)\r\ncompile time:May 20 2016 15:08:19\r\nOK\r\n'
--------------------
AT+GMR
AT version:1.1.0.0(May 11 2016 18:09:56)
SDK version:1.5.4(baaeaebb)
compile time:May 20 2016 15:08:19
OK

--------------------
 command:	 AT+CWMODE_CUR=1
 timeout:	 500 ms
response:	 b'AT+CWMODE_CUR=1\r\r\n\r\nOK\r\n'
--------------------
AT+CWMODE_CUR=1

OK

--------------------
 command:	 AT+CWMODE_CUR?
 timeout:	 500 ms
response:	 b'AT+CWMODE_CUR?\r\r\n+CWMODE_CUR:1\r\n\r\nOK\r\n'
---------------

In [4]:
print(esp.send("AT+CWJAP?"))

--------------------
 command:	 AT+CWJAP?
 timeout:	 500 ms
response:	 b'AT+CWJAP?\r\r\n+CWJAP:"Wilder\xe2\x80\x99s Edge","70:3a:cb:db:02:db",1,-85\r\n\r\nOK\r\n'
--------------------
AT+CWJAP?
+CWJAP:"Wilder’s Edge","70:3a:cb:db:02:db",1,-85

OK



In [71]:
resp =  esp.get_wifi_APs() 

--------------------
 command:	 AT+CWLAP
 timeout:	 5000 ms
..response:	 b'AT+CWLAP\r\r\n+CWLAP:(3,"Wilder\xe2\x80\x99s Edge",-66,"7c:2e:bd:8a:18:61",1,13,0)\r\n+CWLAP:(3,"Wilder\xe2\x80\x99s Edge",-45,"70:3a:cb:db:02:db",1,18,0)\r\n+CWLAP:(3,"bermuda10",-84,"70:3a:cb:b4:9b:85",1,10,0)\r\n+CWLAP:(3,"bermuda10",-75,"70:3a:cb:b4:95:e4",1,18,0)\r\n+CWLAP:(3,"bermuda10",-90,"44:07:0b:0f:13:4f",1,18,0)\r\n+CWLAP:(3,"Wilder\xe2\x80\x99s Edge",-50,"7c:2e:bd:8a:13:d3",6,13,0)\r\n+CWLAP:(3,"Wilder\xe2\x80\x99s Edge",-66,"70:3a:cb:db:06:f0",6,13,0)\r\n+CWLAP:(3,"ATT9ssH4wv_2GEXT",-88,"8c:3b:ad:1f:2c:fc",10,0,0)\r\n+CWLAP:(3,"Wilder\xe2\x80\x99s Edge",-81,"7c:2e:bd:8a:21:51",11,16,0)\r\n+CWLAP:(3,"Wilder\xe2\x80\x99s Edge",-82,"44:07:0b:06:e8:d6",11,15,0)\r\n\r\nOK\r\n'
--------------------


In [76]:
aps = [r for r in resp.split("\r\n") if "+CWLAP:" in r]
print(aps)

['+CWLAP:(3,"Wilder\u2019s Edge",-66,"7c:2e:bd:8a:18:61",1,13,0)', '+CWLAP:(3,"Wilder\u2019s Edge",-45,"70:3a:cb:db:02:db",1,18,0)', '+CWLAP:(3,"bermuda10",-84,"70:3a:cb:b4:9b:85",1,10,0)', '+CWLAP:(3,"bermuda10",-75,"70:3a:cb:b4:95:e4",1,18,0)', '+CWLAP:(3,"bermuda10",-90,"44:07:0b:0f:13:4f",1,18,0)', '+CWLAP:(3,"Wilder\u2019s Edge",-50,"7c:2e:bd:8a:13:d3",6,13,0)', '+CWLAP:(3,"Wilder\u2019s Edge",-66,"70:3a:cb:db:06:f0",6,13,0)', '+CWLAP:(3,"ATT9ssH4wv_2GEXT",-88,"8c:3b:ad:1f:2c:fc",10,0,0)', '+CWLAP:(3,"Wilder\u2019s Edge",-81,"7c:2e:bd:8a:21:51",11,16,0)', '+CWLAP:(3,"Wilder\u2019s Edge",-82,"44:07:0b:06:e8:d6",11,15,0)']


<!--NAVIGATION-->
< [7.4 Stepper Motor Control via I2C](https://jckantor.github.io/cbe61622/07.04-Stepper_Motor_Control_via_I2C.html) | [Contents](toc.html) | [8.0 Raspberry Pi](https://jckantor.github.io/cbe61622/08.00-Raspberry-Pi.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/07.10-Wifi-with-ESP-01.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe61622/07.10-Wifi-with-ESP-01.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>